In [ ]:
from google.colab import drive
import os

gdrive_path='/content/gdrive/MyDrive/Bertopic/shared_work/'

# This will mount your google drive under 'MyDrive'
drive.mount('/content/gdrive', force_remount=True)
# In order to access the files in this notebook we have to navigate to the correct folder
os.chdir(gdrive_path)
# Check manually if all files are present
print(sorted(os.listdir()))

Mounted at /content/gdrive
['BERTopic.ipynb', 'Data-PreProcessing.ipynb', 'Dataset_Statistics.ipynb', 'wasabi_artist_new_dartists.csv', 'wasabi_english_songs.csv', 'wasabi_new_dartists_balanced.csv', 'wasabi_new_songs_english_balanced_artists.csv']


In [ ]:
# importing Songs dataset
import pandas as pd

dsongs = pd.read_csv('/content/gdrive/MyDrive/Bertopic/Wasabi/wasabi_songs.csv', sep='\t', low_memory=False)
list(dsongs.columns)

['Unnamed: 0',
 '_id',
 'abstract',
 'albumTitle',
 'album_genre',
 'animux_content',
 'animux_contents',
 'animux_path',
 'animux_paths',
 'arousal',
 'arousal_predicted',
 'artist',
 'artist_lower',
 'availableCountries',
 'award',
 'begin',
 'bpm',
 'chords_metadata',
 'deezer_mapping',
 'disambiguation',
 'end',
 'explicit_content_lyrics',
 'explicit_content_lyrics_predicted',
 'format',
 'gain',
 'genre',
 'has_emotion_tags',
 'has_social_tags',
 'id_album',
 'id_album_deezer',
 'id_artist_deezer',
 'id_song_deezer',
 'id_song_musicbrainz',
 'isClassic',
 'isrc',
 'langdetect_robust',
 'language',
 'language_detect',
 'lastfm_id',
 'length',
 'lengthAlbum',
 'multitrack_file',
 'multitrack_path',
 'newPreview',
 'notes',
 'origin',
 'position',
 'preview',
 'producer',
 'publicationDate',
 'publicationDateAlbum',
 'rank',
 'rdf',
 'recordLabel',
 'recorded',
 'releaseDate',
 'runtime',
 'subject',
 'title',
 'title_accent_fold',
 'title_lower',
 'urlAllmusic',
 'urlAmazon',
 'urlD

In [ ]:
# Selecting only English language songs
dsongs_English = dsongs[dsongs['language_detect'] == 'english']

# Selecting columns we need for songs dataset, we can add other columns or remove current in the future
column_list_for_english_songs = ['artist', 'genre', 'language', 'language_detect', 'title', 'explicit_content_lyrics_predicted']
dsongs_English = dsongs_English[column_list_for_english_songs]
print(len(dsongs_English))

# Saving this file in Datasets folder
dsongs_English.to_csv('/content/gdrive/MyDrive/Bertopic/shared_work/wasabi_english_songs.csv')

1314253


In [ ]:
# Imporint artists dataset
import pandas as pd

dartists = pd.read_csv('/content/gdrive/MyDrive/Bertopic/Wasabi/wasabi_artists.csv', low_memory=False)
list(dartists.columns)

['_id',
 'disambiguation',
 'endArea',
 'endArea.disambiguation',
 'endArea.id',
 'endArea.name',
 'gender',
 'genres',
 'id_artist_musicbrainz',
 'labels',
 'lifeSpan',
 'lifeSpan.begin',
 'lifeSpan.end',
 'lifeSpan.ended',
 'location',
 'location.city',
 'location.country',
 'location.id_city_musicbrainz',
 'locationInfo',
 'members',
 'name',
 'nameVariations_fold',
 'name_accent_fold',
 'rdf',
 'type',
 'urlAllmusic',
 'urlAmazon',
 'urlDiscogs',
 'urlFacebook',
 'urlITunes',
 'urlMusicBrainz',
 'urlMySpace',
 'urlOfficialWebsite',
 'urlPureVolume',
 'urlRateYourMusic',
 'urlSoundCloud',
 'urlSpotify',
 'urlTwitter',
 'urlWikia',
 'urlWikipedia',
 'urlYouTube',
 'id_artist_discogs',
 'abstract',
 'nameVariations',
 'urls',
 'deezerFans',
 'id_artist_deezer',
 'picture',
 'picture.big',
 'picture.medium',
 'picture.small',
 'picture.standard',
 'picture.xl',
 'urlDeezer',
 'urlWikidata',
 'associatedMusicalArtist',
 'dbp_abstract',
 'dbp_genre',
 'recordLabel',
 'subject',
 'members

In [ ]:
# Removing entries where Gender is not Male, Female or Other
dartists_cleaned = dartists.dropna(subset=['gender'])
dartists_cleaned[["gender"]].value_counts()

gender
Male      14309
Female     5881
Other         8
dtype: int64

In [ ]:
# Removing entries where members column in artist is empty []
dartists_cleaned_solo = dartists_cleaned[dartists_cleaned['members'] == '[]']

# Selecting columns we need for artists dataset, we can add other columns or remove current in the future
column_list_for_dartists = ['name', 'gender']
dartists_cleaned_solo = dartists_cleaned_solo[column_list_for_dartists]

In [ ]:
# Checking new artist dataset where gender and solo artists filtered and chosen
dartists_cleaned_solo[["gender"]].value_counts()

gender
Male      8772
Female    4498
Other        6
dtype: int64

In [ ]:
# Choosing artist name from songs dataset with english language and removing the other ones
# (the ones that does not sing in english, because in artist it is not specified fully in what language artist sings, we went forward from Songs to Artists)
idxs = list(dsongs_English.artist.values)

dartists_cleaned_solo = dartists_cleaned_solo[pd.Series(list(dartists_cleaned_solo.name), index=dartists_cleaned_solo.index).isin(idxs)]

In [ ]:
# Checking new artist dataset where english singing artists are kept
dartists_cleaned_solo[["gender"]].value_counts()

gender
Male      6890
Female    3664
Other        6
dtype: int64

In [ ]:
# Saving the file for artists datasrt
dartists_cleaned_solo.to_csv('/content/gdrive/MyDrive/Bertopic/shared_work/wasabi_artist_new_dartists.csv')

In [ ]:
df_male = dartists_cleaned_solo[dartists_cleaned_solo['gender'] == 'Male']
df_female = dartists_cleaned_solo[dartists_cleaned_solo['gender'] == 'Female']

# Determine the smaller size among the two groups
min_size = min(len(df_male), len(df_female))

# Truncate both datasets to have the same size
df_male_balanced = df_male.sample(min_size)
df_female_balanced = df_female.sample(min_size)

# Combine the balanced datasets
df_balanced = pd.concat([df_male_balanced, df_female_balanced])
df_balanced.to_csv('/content/gdrive/MyDrive/Bertopic/shared_work/wasabi_new_dartists_balanced.csv')

In [ ]:
df_balanced[["gender"]].value_counts()

gender
Female    3664
Male      3664
dtype: int64

In [ ]:
print(len(dsongs_English))

1314253


In [ ]:
# Again removing songs which does not belong to artists in wasabi_new_dartists_balanced.csv file
idxs2 = list(df_balanced.name.values)

dsongs2_English = dsongs_English[pd.Series(list(dsongs_English.artist), index=dsongs_English.index).isin(idxs2)]

In [ ]:
print(len(dsongs2_English))

176616


In [ ]:
dsongs2_English.to_csv('/content/gdrive/MyDrive/Bertopic/shared_work/wasabi_new_songs_english_balanced_artists.csv')

In [ ]:
import requests

class MusixMatch:
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = "http://api.musixmatch.com/ws/1.1/"

    def _make_request(self, method, params):
        params['apikey'] = self.api_key
        response = requests.get(self.base_url + method, params=params)
        response.raise_for_status()
        return response.json()

    def search_track(self, track_name):
        method = 'track.search'
        params = {'q_track': track_name, 'page_size': 1, 'page': 1, 's_track_rating': 'desc'}
        data = self._make_request(method, params)
        track_list = data['message']['body']['track_list']
        return track_list[0]['track']['track_id'] if track_list else None

    def get_lyrics(self, track_id):
        method = 'track.lyrics.get'
        params = {'track_id': track_id}
        data = self._make_request(method, params)
        return data['message']['body']['lyrics']['lyrics_body'] if data['message']['body'] else None

api_key = '40cf7e6a142e0cf7a88b3abcc1d06969'
musixmatch = MusixMatch(api_key)

# Search for a track and get its lyrics
track_name = "Shape of You"
track_id = musixmatch.search_track(track_name)
if track_id:
    lyrics = musixmatch.get_lyrics(track_id)
    print("Lyrics:\n", lyrics)
else:
    print("Track not found.")

# get the lyrics

Lyrics:
 The club isn't the best place to find a lover
So the bar is where I go (mm-mm)
Me and my friends at the table doing shots
Drinking fast, and then we talk slow (mm-mm)
You come over and start up a conversation with just me
And trust me, I'll give it a chance now (mm-mm)
Take my hand, stop, put "Van the Man" on the jukebox
And then we start to dance
And now I'm singing like

Girl, you know I want your love
Your love was handmade for somebody like me
Come on now, follow my lead
I may be crazy, don't mind me
Say, "Boy, let's not talk too much
Grab on my waist and put that body on me"
Come on now, follow my lead
Come, come on now, follow my lead
(Mm-mm)

I'm in love with the shape of you
We push and pull like a magnet do
Although my heart is falling too
I'm in love with your body
Last night you were in my room
And now my bed sheets smell like you
Every day discovering something brand new
Oh, I'm in love with your body

Oh I, oh I, oh I, oh I
Oh, I'm in love with your body
Oh I, oh 

In [ ]:
def append_lyrics_to_csv(csv_file_path, api_key, start_row=0):
    musixmatch = MusixMatch(api_key)
    df = pd.read_csv(csv_file_path)

    # Add a 'lyrics_addbyus' column if it doesn't exist
    if 'lyrics_addbyus' not in df.columns:
        df['lyrics_addbyus'] = pd.NA

    for i, row in df.iterrows():
        if i < start_row:
            continue  # Skip rows until the starting row is reached

        # Check if lyrics are already present
        if pd.isna(df.at[i, 'lyrics_addbyus']):
            print(i)
            track_name = row['title']  # Replace 'title' with your actual column name if different
            track_id = musixmatch.search_track(track_name)
            if track_id:
                lyrics = musixmatch.get_lyrics(track_id)
                df.at[i, 'lyrics_addbyus'] = lyrics  # Add lyrics to the DataFrame
            else:
                df.at[i, 'lyrics_addbyus'] = "Lyrics Not Found"

            # Save progress after each update directly to the same CSV file
            df.to_csv(csv_file_path, index=False)

# Example usage
csv_file_path = '/content/gdrive/MyDrive/Bertopic/shared_work/wasabi_new_songs_english_balanced_artists.csv'
api_key = 'b75a61e15329efb843009eece12d080f'  # Your Musixmatch API key
start_row = 8276  # Set the row number from which you want to start fetching lyrics, change it everytime you ran the code
append_lyrics_to_csv(csv_file_path, api_key, start_row)


In [ ]:
dsongs3_English = pd.read_csv("wasabi_new_songs_english_balanced_artists.csv")  # The file with artist names
# Initialize a new column for gender in df_artists
dsongs3_English['gender'] = pd.NA

# Iterate over each artist in final song lyrics file
for index, row in dsongs3_English.iterrows():
    artist_name = row['artist']

    # Find the matching artist in artist final file and get the gender
    gender_row = df_balanced[df_balanced['name'] == artist_name]  # Replace 'artist_name' with the actual column name in df_gender
    if not gender_row.empty:
        dsongs3_English.at[index, 'gender'] = gender_row.iloc[0]['gender']  # Replace 'gender' with the actual column name in df_gender
    else:
        dsongs3_English.at[index, 'gender'] = "Unknown"  # Or any other placeholder for missing gender


<ipython-input-27-a3d080d0a04e>:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  dsongs3_English = pd.read_csv("wasabi_new_songs_english_balanced_artists.csv")  # The file with artist names


In [ ]:
dsongs3_English.to_csv("updated_artists_with_gender_lyrics.csv", index=False)